# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2806,-36.5098,9.12,91,100,5.30,GS,1707836414
1,1,papao,12.6919,-16.3792,38.06,34,0,2.06,SN,1707836416
2,2,las tablas,7.7678,-80.2764,27.26,99,100,6.53,PA,1707836418
3,3,amga,60.8942,131.9800,-28.55,99,100,1.56,RU,1707836421
4,4,bredasdorp,-34.5331,20.0428,25.34,48,17,7.48,ZA,1707836422


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", frame_width=800, frame_height=800, scale=1, size="Humidity", color="City")
# Display the map

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = city_data_df[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values

ideal_weather_df = ideal_weather_df.dropna()
# Display sample data

ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,91,zeribet el oued,34.6830,6.5047,22.15,28,0,0.88,DZ,1707836512
272,272,rouissat,31.9240,5.3498,24.04,33,0,2.06,DZ,1707836614
273,273,woolgoolga,-30.1108,153.2000,22.93,84,0,3.13,AU,1707836615
341,341,chipinge,-20.1889,32.6194,24.57,50,0,2.12,ZW,1707836664
346,346,lihue,21.9770,-159.3688,21.66,85,0,3.60,US,1707836678


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
91,zeribet el oued,DZ,34.6830,6.5047,28,
272,rouissat,DZ,31.9240,5.3498,33,
273,woolgoolga,AU,-30.1108,153.2000,84,
341,chipinge,ZW,-20.1889,32.6194,50,
346,lihue,US,21.9770,-159.3688,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
   
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = 'category:hotels'
    params["bias"] = f'proximity:{lat},{lon}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
zeribet el oued - nearest hotel: No hotel found
rouissat - nearest hotel: No hotel found
woolgoolga - nearest hotel: No hotel found
chipinge - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
talcahuano - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: No hotel found
mohr - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
91,zeribet el oued,DZ,34.6830,6.5047,28,No hotel found
272,rouissat,DZ,31.9240,5.3498,33,No hotel found
273,woolgoolga,AU,-30.1108,153.2000,84,No hotel found
341,chipinge,ZW,-20.1889,32.6194,50,No hotel found
346,lihue,US,21.9770,-159.3688,85,No hotel found
357,talcahuano,CL,-36.7145,-73.1141,46,No hotel found
498,brisas de zicatela,MX,15.8367,-97.0429,65,No hotel found
556,mohr,IR,27.5515,52.8821,49,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
mat_plot2 = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles='OSM', frame_width = 800, frame_height = 800, hover_cols = ["Hotel Name", "Country"] )
# Display the map
# YOUR CODE HERE
mat_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)